# OLM Docs FAQ/Slack Bot


## Import dependencies and credentials

In [1]:
# pip install langchain langchain-openai python-dotenv nltk openai chromadb tiktoken

In [67]:
import uuid
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings

import re
import requests

from chromadb import HttpClient
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import MarkdownTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_openai import ChatOpenAI
from sentence_transformers import SentenceTransformer

from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import time
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from dotenv import load_dotenv, find_dotenv

### Step 1:

Only if using OpenAI, in the `credentials.env` file in the folder, change the XXX value for the `OPENAI_API_KEY` to your actual OPENAI API KEY

In [3]:
# load_dotenv(find_dotenv("credentials.env"), override=True)

## Read the Dataset and create Vector Store

In [5]:
loader = DirectoryLoader('../data/external/olm/docs/', glob="**/*.md", loader_cls=TextLoader)
documents = loader.load()

In [6]:
## Verification
'../data/external/olm/docs/slack_samples.md' in [i.metadata['source'] for i in documents]

True

In [7]:
## Split the documents into chunks. Is there a better way than hardcoding size as 1000?
text_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

436

In [9]:
Chroma().delete_collection()

In [10]:
# If using OpenAI embeddings
# embeddings = OpenAIEmbeddings()
# docsearch = Chroma.from_documents(texts, embeddings, persist_directory='database')

In [11]:
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="avsolatorio/GIST-Embedding-v0")
e = SentenceTransformerEmbeddings(model_name="avsolatorio/GIST-Embedding-v0")

client = chromadb.PersistentClient()
collection = client.get_or_create_collection("collection", embedding_function=embedding_func)

if collection.count() < 1:
    print("populating db")

    for doc in texts:
        collection.add(
            ids=[str(uuid.uuid1())],
            metadatas=doc.metadata, 
            documents=doc.page_content
            )


db = Chroma(client=client,
            collection_name="collection",
            embedding_function=e
    )

docsearch = db.as_retriever(threshold=0.75)

In [12]:
docsearch

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x2ac40fcd0>)

In [13]:
qa = pd.read_csv("../data/test/sample_qna.csv")

In [14]:
qa.head()

,Question,Answer
0,"I have an operator catalog image with me, `qua...",You can make the operators available for insta...
1,"I added a catalog source to my cluster, but I ...","Once you add a catalog source to your cluster,..."
2,How can I see what operators are available for...,You can see what operators are available for i...
3,How can I install an operator on my cluster fr...,You can see what operators are available for i...
4,"Hey, looking for guidance on how to do Tier 2 ...",You're going to want to avoid a channel called...


In [15]:
questions = qa['Question'].tolist()
real_answers = qa['Answer'].tolist()
generated_answers = list()

In [16]:
selected_questions = questions[0:2]

## Model for QnA (using Mistral 7B model deployed locally using llama CPP

To deploy locally
`python3 -m llama_cpp.server --model mistral-7b-instruct-v0.1.Q4_K_M.gguf`

In [18]:
llm = ChatOpenAI(base_url="http://localhost:8000/v1", 
                 api_key="x",)

In [19]:
# llm = ChatOpenAI()

## Without RAG - Basic Prompt

In [52]:
template = """Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

In [53]:
def answer_question(query, runnable):
    """
    Takes in query, and llm chain to generate answer
    """
    ## Generate answer
    answer = runnable.invoke({"question": query})
    return answer

In [54]:
generated_answers_1 = list()
i=1
for query in selected_questions:

    answer = answer_question(query, chain)

    generated_answers_1.append(answer.content)
    print(f"Q: {query}\nA: {answer}\n")

    ## Add delay to avoid rate limit error
    time.sleep(1)
    print(f"{i}/{len(selected_questions)} done")
    i+=1

Q: I have an operator catalog image with me, `quay.io/operator-framework/upstream-community-operators:latest`, how do I make the operators included in the catalog image, available for installation on my cluster? 


A: content=' To use the operators included in the `quay.io/operator-framework/upstream-community-operators:latest` container image on your cluster, you need to follow these steps:\n\n1. Pull the container image onto your cluster using the kubectl command-line tool:\n```\nkubectl pull quay.io/operator-framework/upstream-community-operators:latest\n```\n2. Load the container image into the default namespace of your cluster using the kubectl command-line tool:\n```\nkubectl load --local --image=quay.io/operator-framework/upstream-community-operators:latest --namespace=default\n```\n3. Verify that the container image has been loaded into your cluster by running a kubectl get pods command in the default namespace:\n```\nkubectl get pods -n default\n```\nThis should return a list 

### Without RAG - Advanced Prompt

In [55]:
template = """You are a support engineer who is trying to generate answers for questions around the Operator Frameworks product. Your goal is to answer questions that OLM customers and users would find relevant, informative, and useful. You should be descriptive and provide links to support your answer.

Here is a description of the product:
Operator Lifecycle Manager (OLM) project is a component of the Operator Frameworkan open source toolkit to manage Kubernetes native applications, called Operators, in a streamlined and scalable way.

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

In [56]:
def answer_question(query, runnable):
    """
    Takes in query, and llm chain to generate answer
    """
    ## Generate answer
    answer = runnable.invoke({"question": query})
    return answer

In [57]:
generated_answers_2 = list()
i=1
for query in selected_questions:

    answer = answer_question(query, chain)

    generated_answers_2.append(answer.content)
    print(f"Q: {query}\nA: {answer}\n")

    ## Add delay to avoid rate limit error
    time.sleep(1)
    print(f"{i}/{len(selected_questions)} done")
    i+=1

Q: I have an operator catalog image with me, `quay.io/operator-framework/upstream-community-operators:latest`, how do I make the operators included in the catalog image, available for installation on my cluster? 


A: content=" To make the operators included in the `quay.io/operator-framework/upstream-community-operators:latest` catalog image available for installation on your cluster, you can follow these steps:\n\n1. First, you need to import the catalog image into your OLM cluster using the `olmctl import` command.\n```css\nolmctl import --namespace=default --source=quay://operator-framework/upstream-community-operators:latest\n```\nThis command will import the catalog image into the `default` namespace of your OLM cluster.\n\n2. Once the catalog image has been imported, you can view the available operators in the catalog by running the `olmctl list` command. This command will list all the operators in the catalog along with their versions and other metadata.\n```css\nolmctl list --

### With RAG

In [58]:
template = """
You are a support engineer who is trying to generate answers for questions around the Operator Frameworks product. Your goal is to answer questions that OLM customers and users would find relevant, informative, and useful. You should be descriptive and provide links to support your answer.

Here is a description of the product:
Operator Lifecycle Manager (OLM) project is a component of the Operator Framework open source toolkit to manage Kubernetes native applications, called Operators, in a streamlined and scalable way.

Question: {question}

Here are a few input and output pairs examples to guide the model:

Input: "What is OLM?"
Output: "Operator Lifecycle Manager (OLM) project is a component of the Operator Frameworkan open source toolkit to manage Kubernetes native applications, called Operators, in a streamlined and scalable way."

Input: "What are OLM features?"
Output: "OLM provides rich update mechanisms to keep Kubernetes native applications up to date automatically. With OLMs packaging format Operators can express dependencies on the platform and on other Operators."

Please help this OLM customer to find the answer to this {question} given the relevant content around the question {context}:

If you don't know the answer, please respond with: "I'm sorry, I don't have enough information to generate that content."
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

In [59]:
def answer_question(query, index, chain):
    """
    Takes in query, index to search from, and llm chain to generate answer
    """
    ## Retrieve docs

    docs = index.get_relevant_documents(query)
    print(docs)
    print(len(docs))
    
    ## Generate answer
    print(index)
    answer = chain.invoke({"question": query, "context": index})
    return answer

In [60]:
generated_answers_3 = list()
i=1
for query in selected_questions:
    answer = answer_question(query, docsearch, chain)

    generated_answers_3.append(answer.content)
    print(f"Q: {query}\nA: {answer}\n")

    ## Add delay to avoid rate limit error
    time.sleep(1)
    print(f"{i}/{len(selected_questions)} done")
    i+=1

[Document(page_content='There are many possible ways to build a catalog, but an extremely simple approach would be to:\n\n- Maintain a single configuration file containing image references for each operator in the catalog\n   ```yaml\n   name: community-operators\n   repo: quay.io/community-operators/catalog\n   tag: latest\n   references:\n   - name: etcd-operator\n     image: quay.io/etcd-operator/catalog@sha256:5891b5b522d5df086d0ff0b110fbd9d21bb4fc7163af34d08286a2e846f6be03\n   - name: prometheus-operator\n     image: quay.io/prometheus-operator/catalog@sha256:e258d248fda94c63753607f7c4494ee0fcbe92f1a76bfdac795c9d84101eb317', metadata={'source': '../data/external/olm/docs/Reference/file-based-catalogs.md'}), Document(page_content='As a cluster administrator, you can install an Operator from the OperatorHub using the OpenShift Container Platform web console or the CLI. You can then subscribe the Operator to one or more namespaces to make it available for developers on your cluster.\

# Evaluate the generated answers using a metric

In [62]:
def calc_bleu(reference, candidate):

    for i in range(len(reference)):
        print('BLEU score -> {}'.format(sentence_bleu(reference[i].split(), candidate[i].split(), smoothing_function=SmoothingFunction().method4)))

In [63]:
calc_bleu(real_answers[0:2], generated_answers_1)

BLEU score -> 0.0026912080031739434
BLEU score -> 0


In [64]:
calc_bleu(real_answers[0:2], generated_answers_2)

BLEU score -> 0.0016114342264778043
BLEU score -> 0


In [65]:
calc_bleu(real_answers[0:2], generated_answers_3)

BLEU score -> 0.0018364395093896158
BLEU score -> 0.0009457964768925455


# Post processing or Sanity Checks

In [66]:
def check_links(text):
    urls = re.findall(r'\((https?:\/\/[^\s\/$.?#].[^\s()]*\/[^\s\/$.?#]*(?:\.html)?)\)', text)
    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                print(f"The link {url} is valid.")
            else:
                print(f"The link {url} returned a status code of {response.status_code}.")
        except requests.exceptions.RequestException as e:
            print(f"The link {url} could not be reached. Error: {e}")

print("generated_answers_1 returned")
for text in generated_answers_1:
    check_links(text)

print("generated_answers_2 returned")
for text in generated_answers_2:
    check_links(text)

print("generated_answers_3 returned")
for text in generated_answers_3:
    check_links(text)

generated_answers_1 returned
generated_answers_2 returned
generated_answers_3 returned
